In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.constants import codata

In [2]:
path=r"C:\Users\cjang\Desktop\pyeis\PyEIS\Tutorials\data\\"
data = ['ex1.mpt']

In [3]:

def push(path, data, cycle='off', mask=['none','none']):
    df_raw0 = []
    cycleno = []
    for j in range(len(data)):
        if data[j].find(".mpt") != -1: #file is a .mpt file
            df_raw0.append(extract_mpt(path=path, EIS_name=data[j])) #reads all datafiles
        elif data[j].find(".DTA") != -1: #file is a .dta file
            df_raw0.append(extract_dta(path=path, EIS_name=data[j])) #reads all datafiles
        elif data[j].find(".z") != -1: #file is a .z file
            df_raw0.append(extract_solar(path=path, EIS_name=data[j])) #reads all datafiles
        else:
            print('Data file(s) could not be identified')

        cycleno.append(df_raw0[j].cycle_number)
        if np.min(cycleno[j]) <= np.max(cycleno[j-1]):
            if j > 0: #corrects cycle_number except for the first data file
                df_raw0[j].update({'cycle_number': cycleno[j]+np.max(cycleno[j-1])}) #corrects cycle number

    df_raw = [i for i in df_raw0][0]
    return df_raw



#TAKEN FROM PYEIS LIBRARY
def correct_text_EIS(text_header):
    '''Corrects the text of '*.mpt' and '*.dta' files into readable parameters without spaces, ., or /
    
    <E_we> = averaged Wew value for each frequency
    <I> = Averaged I values for each frequency
    |E_we| = module of Ewe
    |I_we| = module of Iwe
    Cs/F = Capacitance caluculated using an R+C (series) equivalent circuit
    Cp/F = Capacitance caluculated using an R-C (parallel) equivalent circuit
    Ref.:
        - EC-Lab User's Manual
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    if text_header == 'freq/Hz' or text_header == '  Freq(Hz)':
        return 'f'
    elif text_header == 'Re(Z)/Ohm' or text_header == "Z'(a)":
        return 're'
    elif text_header == '-Im(Z)/Ohm' or text_header == "Z''(b)":
        return 'im'
#    elif text_header == "Z''(b)":
#        return 'im_neg'
    elif text_header == '|Z|/Ohm':
        return 'Z_mag'
    elif text_header == 'Phase(Z)/deg':
        return 'Z_phase'
    elif text_header == 'time/s' or text_header == 'Time(Sec)':
        return 'times'
    elif text_header == '<Ewe>/V' or text_header == 'Bias':
        return 'E_avg'
    elif text_header == '<I>/mA':
        return 'I_avg'
    elif text_header == 'Cs/F':
        return 'Cs' ####
    elif text_header == 'Cp/F':
        return 'Cp'
    elif text_header == 'cycle number':
        return 'cycle_number'
    elif text_header == 'Re(Y)/Ohm-1':
        return 'Y_re'
    elif text_header == 'Im(Y)/Ohm-1':
        return 'Y_im'
    elif text_header == '|Y|/Ohm-1':
        return 'Y_mag'
    elif text_header == 'Phase(Y)/deg':
        return 'Y_phase'
    elif text_header == 'Time':
        return 'times'
    elif text_header == 'Freq':
        return 'f'
    elif text_header == 'Zreal':
        return 're'
    elif text_header == 'Zimag':
        return 'im'
    elif text_header == 'Zmod':
        return 'Z_mag'
    elif text_header == 'Vdc':
        return 'E_avg'
    elif text_header == 'Idc':
        return 'I_avg'
    elif text_header == 'I/mA':
        return 'ImA'
    elif text_header == 'Ewe/V':
        return 'EweV'
    elif text_header == 'half cycle':
        return 'half_cycle'
    elif text_header == 'Ns changes':
        return 'Ns_changes'
    else:
        return text_header


#TAKEN FROM PYEIS LIBRARY
def extract_mpt(path, EIS_name):
    '''
    Extracting PEIS and GEIS data files from EC-lab '.mpt' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    EIS_init = pd.read_csv(path+EIS_name, sep='\t', nrows=1,header=0,names=['err'], encoding='latin1') #findes line that states skiplines
#    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20])-1, encoding='latin1') #locates number of skiplines
    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1])-1, encoding='latin1') #locates number of skiplines
    names_EIS = []
    for j in range(len(EIS_test_header_names.columns)):
        names_EIS.append(correct_text_EIS(EIS_test_header_names.columns[j])) #reads coloumn text
#    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20]), names=names_EIS, encoding='latin1')
    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1]), names=names_EIS, encoding='latin1')


#TAKEN FROM PYEIS LIBRARY
def extract_dta(path, EIS_name):
    '''
    Extracting data files from Gamry '.DTA' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('ZCURVE')+1  ##ZC.get_loc('ZCURVE')+3
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=ZC.get_loc('ZCURVE')+3, names=header_names, encoding='latin1')
    data.update({'im': np.abs(data.im)})
    data = data.assign(cycle_number = 1.0)
    return data


#TAKEN FROM PYEIS LIBRARY
def extract_solar(path, EIS_name):
    '''
    Extracting data files from Solartron's '.z' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('  Freq(Hz)')
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc+2, names=header_names, encoding='latin1')
    data.update({'im': -data.im})
    data = data.assign(cycle_number = 1.0)
    return data

#
#print()
#print('---> Data Extraction Script Loaded (v. 0.0.2 - 06/27/18)')

In [4]:
push(path,data)

,f,re,im,Z_mag,Z_phase,times,E_avg,I_avg,Cs/µF,Cp/µF,cycle_number,I Range,|Ewe|/V,|I|/A,Y_re,Y_im,Y_mag,Y_phase,Unnamed: 18
0,200019.480000,428.90558,372.31183,567.95782,-40.959641,13125.243035,-1.050056,-0.003182,0.002137,0.000918,1.0,12,0.009270,1.632094e-05,0.001330,0.001154,0.001761,40.959641,NaN
1,149677.700000,389.09244,779.30804,871.04187,-63.467991,13125.926037,-1.050440,-0.003601,0.001364,0.001092,1.0,12,0.010166,1.167101e-05,0.000513,0.001027,0.001148,63.467991,NaN
2,112011.700000,790.36896,1276.19290,1501.11670,-58.229328,13126.609058,-1.050325,-0.003597,0.001113,0.000805,1.0,12,0.010198,6.793900e-06,0.000351,0.000566,0.000666,58.229328,NaN
3,83828.102000,1450.48320,1320.92870,1961.82420,-42.323559,13127.165037,-1.050292,-0.003622,0.001437,0.000652,1.0,12,0.010902,5.557279e-06,0.000377,0.000343,0.000510,42.323559,NaN
4,62734.359000,1863.74980,1040.74610,2134.64650,-29.179655,13127.722056,-1.050266,-0.003596,0.002438,0.000579,1.0,12,0.010428,4.884899e-06,0.000409,0.000228,0.000468,29.179655,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,0.031914,37919.22300,26275.43000,46133.12900,-34.719315,15062.747178,-1.050242,-0.003194,189.798080,61.569504,2.0,14,0.010236,2.218715e-07,0.000018,0.000012,0.000022,34.719315,NaN
114,0.023877,46010.07000,29608.37700,54713.64100,-32.762070,15209.504636,-1.050244,-0.003157,225.127670,65.927544,2.0,14,0.010234,1.870528e-07,0.000015,0.000010,0.000018,32.762070,NaN
115,0.017841,51408.21100,32122.89500,60619.17600,-31.999582,15405.897438,-1.050247,-0.003112,277.704280,77.981552,2.0,14,0.010235,1.688474e-07,0.000014,0.000009,0.000016,31.999582,NaN
116,0.013378,64626.45300,32967.96500,72549.74200,-27.027544,15667.806481,-1.050241,-0.003047,360.870120,74.518387,2.0,14,0.010233,1.410469e-07,0.000012,0.000006,0.000014,27.027544,NaN


In [ ]:
def correct_text_EIS(text_header):
    '''Corrects the text of '*.mpt' and '*.dta' files into readable parameters without spaces, ., or /
    
    <E_we> = averaged Wew value for each frequency
    <I> = Averaged I values for each frequency
    |E_we| = module of Ewe
    |I_we| = module of Iwe
    Cs/F = Capacitance caluculated using an R+C (series) equivalent circuit
    Cp/F = Capacitance caluculated using an R-C (parallel) equivalent circuit
    Ref.:
        - EC-Lab User's Manual
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    if text_header == 'freq/Hz' or text_header == '  Freq(Hz)':
        return 'f'
    elif text_header == 'Re(Z)/Ohm' or text_header == "Z'(a)":
        return 're'
    elif text_header == '-Im(Z)/Ohm' or text_header == "Z''(b)":
        return 'im'
#    elif text_header == "Z''(b)":
#        return 'im_neg'
    elif text_header == '|Z|/Ohm':
        return 'Z_mag'
    elif text_header == 'Phase(Z)/deg':
        return 'Z_phase'
    elif text_header == 'time/s' or text_header == 'Time(Sec)':
        return 'times'
    elif text_header == '<Ewe>/V' or text_header == 'Bias':
        return 'E_avg'
    elif text_header == '<I>/mA':
        return 'I_avg'
    elif text_header == 'Cs/F':
        return 'Cs' ####
    elif text_header == 'Cp/F':
        return 'Cp'
    elif text_header == 'cycle number':
        return 'cycle_number'
    elif text_header == 'Re(Y)/Ohm-1':
        return 'Y_re'
    elif text_header == 'Im(Y)/Ohm-1':
        return 'Y_im'
    elif text_header == '|Y|/Ohm-1':
        return 'Y_mag'
    elif text_header == 'Phase(Y)/deg':
        return 'Y_phase'
    elif text_header == 'Time':
        return 'times'
    elif text_header == 'Freq':
        return 'f'
    elif text_header == 'Zreal':
        return 're'
    elif text_header == 'Zimag':
        return 'im'
    elif text_header == 'Zmod':
        return 'Z_mag'
    elif text_header == 'Vdc':
        return 'E_avg'
    elif text_header == 'Idc':
        return 'I_avg'
    elif text_header == 'I/mA':
        return 'ImA'
    elif text_header == 'Ewe/V':
        return 'EweV'
    elif text_header == 'half cycle':
        return 'half_cycle'
    elif text_header == 'Ns changes':
        return 'Ns_changes'
    else:
        return text_header
    
def extract_mpt(path, EIS_name):
    '''
    Extracting PEIS and GEIS data files from EC-lab '.mpt' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    EIS_init = pd.read_csv(path+EIS_name, sep='\t', nrows=1,header=0,names=['err'], encoding='latin1') #findes line that states skiplines
#    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20])-1, encoding='latin1') #locates number of skiplines
    EIS_test_header_names = pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1])-1, encoding='latin1') #locates number of skiplines
    names_EIS = []
    for j in range(len(EIS_test_header_names.columns)):
        names_EIS.append(correct_text_EIS(EIS_test_header_names.columns[j])) #reads coloumn text
#    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:20]), names=names_EIS, encoding='latin1')
    return pd.read_csv(path+EIS_name, sep='\t', skiprows=int(EIS_init.err[0][18:-1]), names=names_EIS, encoding='latin1')

def extract_dta(path, EIS_name):
    '''
    Extracting data files from Gamry '.DTA' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('ZCURVE')+1  ##ZC.get_loc('ZCURVE')+3
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=ZC.get_loc('ZCURVE')+3, names=header_names, encoding='latin1')
    data.update({'im': np.abs(data.im)})
    data = data.assign(cycle_number = 1.0)
    return data

def extract_solar(path, EIS_name):
    '''
    Extracting data files from Solartron's '.z' format, coloums are renames following correct_text_EIS()
    
    Kristian B. Knudsen (kknu@berkeley.edu || kristianbknudsen@gmail.com)
    '''
    dummy_col = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I','J','K','L','M','N','O','P']
    init = pd.read_csv(path+EIS_name, encoding='latin1', sep='\t', names=dummy_col)
    ZC = pd.Index(init.A)
    header_loc = ZC.get_loc('  Freq(Hz)')
    
    header_names_raw = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc, encoding='latin1') #locates number of skiplines
    header_names = []
    for j in range(len(header_names_raw.columns)):
        header_names.append(correct_text_EIS(header_names_raw.columns[j])) #reads coloumn text
    data = pd.read_csv(path+EIS_name, sep='\t', skiprows=header_loc+2, names=header_names, encoding='latin1')
    data.update({'im': -data.im})
    data = data.assign(cycle_number = 1.0)
    return data

#
#print()
#print('---> Data Extraction Script Loaded (v. 0.0.2 - 06/27/18)')